In [35]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np

In [36]:
#===============================================
# Small helpers to keep formatting consistent.
#===============================================
hw = {
    "font": 'system-ui , -apple-system , "Segoe UI" , Roboto , Arial' ,
    "text": "#111827" , "muted": "#6B7280" , "bg": "#FFFFFF" , "panel": "#F9FAFB" ,
    "panel2": "#F3F4F6" , "grid": "#E5E7EB" , "accent": "#2563EB" ,
    "radius": "10px" , "pad": "10px" , "width": "900px" ,
}

In [37]:
#========
# Setup
#========
kpi = pd.DataFrame({
    "region": ["East" , "West" , "Central" , "North"] ,
    "sales": [1297 , 1579 , 1321 , 980] ,
    "returns": [58 , 71 , 75 , 22] ,
    "margin": [0.18 , 0.12 , 0.21 , 0.09] ,
})
kpi["return_rate"] = kpi["returns"] / kpi["sales"]
kpi = kpi[["region" , "sales" , "returns" , "return_rate" , "margin"]]
kpi

df = pd.DataFrame({
    "region": ["East" , "East" , "West" , "West"] ,
    "product": ["Cable" , "Wire" , "Cable" , "Wire"] ,
    "month": ["2025-11" , "2025-12" , "2025-11" , "2025-12"] ,
    "sales": [1297 , 1321 , 1579 , 1100] ,
    "returns": [58 , 75 , 71 , 60] ,
})
df

mi = (
    df.set_index(["region" , "product" , "month"]).sort_index()
      .rename_axis(index = ["Region" , "Product" , "Month"])
)
mi

,region,sales,returns,return_rate,margin
0,East,1297,58,0.044719,0.18
1,West,1579,71,0.044965,0.12
2,Central,1321,75,0.056775,0.21
3,North,980,22,0.022449,0.09


,region,product,month,sales,returns
0,East,Cable,2025-11,1297,58
1,East,Wire,2025-12,1321,75
2,West,Cable,2025-11,1579,71
3,West,Wire,2025-12,1100,60


sales  returns
Region Product Month                  
East   Cable   2025-11   1297       58
       Wire    2025-12   1321       75
West   Cable   2025-11   1579       71
       Wire    2025-12   1100       60

In [38]:
#============
# Helpers
#============
def base_light(s: pd.io.formats.style.Styler , caption: str):
    css = [
        {"selector": "html , body" ,
         "props": [("background" , hw["bg"]) , ("margin" , "18px") , ("color" , hw["text"])]} ,

        {"selector": "caption" ,
         "props": [("caption-side" , "top") , ("font-weight" , "800") ,
                   ("font-size" , "18px") , ("color" , hw["text"]) ,
                   ("padding" , "8px 0 12px 0")]} ,

        {"selector": "table" ,
         "props": [("width" , hw["width"]) , ("font-family" , hw["font"]) ,
                   ("color" , hw["text"]) , ("background-color" , hw["bg"]) ,
                   ("border" , f"1px solid {hw['grid']}") ,
                   ("border-radius" , hw["radius"]) , ("border-collapse" , "separate") ,
                   ("border-spacing" , "0") , ("overflow" , "hidden")]} ,

        {"selector": "th" ,
         "props": [("background-color" , hw["panel2"]) , ("padding" , hw["pad"]) ,
                   ("border-bottom" , f"1px solid {hw['grid']}") ,
                   ("text-align" , "left") , ("font-weight" , "700")]} ,

        {"selector": "td" ,
         "props": [("padding" , hw["pad"]) ,
                   ("border-bottom" , f"1px solid {hw['grid']}")]} ,

        {"selector": "tbody tr:last-child td" ,
         "props": [("border-bottom" , "none")]} ,

        {"selector": "tbody tr:hover" ,
         "props": [("background-color" , "#EEF2FF")]} ,
    ]

    return (
        s.set_caption(caption).set_table_styles(css , overwrite = True)
         .set_properties(**{
             "background-color": f"{hw['bg']} !important" ,
             "color": f"{hw['text']} !important" ,
             "font-family": hw["font"] ,
         })
    )

In [39]:
#============
# Helpers
#============
def fmt_kpi(s):
    return (
        s.format({"sales": "{:,.0f}" , "returns": "{:,.0f}"})
         .format({"return_rate": "{:.1%}" , "margin": "{:.1%}"})
         .format(na_rep="—")
    )

def zebra_styles(df_ , even = hw["bg"] , odd = hw["panel"]):
    out = pd.DataFrame("" , index = df_.index , columns = df_.columns)
    out.iloc[::2 , :] = f"background-color: {even} !important;"
    out.iloc[1::2 , :] = f"background-color: {odd} !important;"
    return out

def hex_lerp(c1 , c2 , t):
    c1 = c1.lstrip("#")
    c2 = c2.lstrip("#")
    r1 , g1 , b1 = int(c1[0:2] , 16) , int(c1[2:4] , 16) , int(c1[4:6] , 16)
    r2 , g2 , b2 = int(c2[0:2] , 16) , int(c2[2:4] , 16) , int(c2[4:6] , 16)
    r = int(r1 + (r2 - r1) * t)
    g = int(g1 + (g2 - g1) * t)
    b = int(b1 + (b2 - b1) * t)
    return f"#{r:02x}{g:02x}{b:02x}"

In [40]:
#============================
# Case 1) Clean KPI table
#============================
s1 = base_light(kpi.style.hide(axis = "index") , "Case 1: Clean KPI table")
s1 = fmt_kpi(s1).apply(lambda d: zebra_styles(d) , axis = None)
s1

region,sales,returns,return_rate,margin
East,1297,58,0.044719,0.180000
West,1579,71,0.044965,0.120000
Central,1321,75,0.056775,0.210000
North,980,22,0.022449,0.090000


In [41]:
#===============================================
# Case 2) Status badges (Good / Watch / Risk)
#===============================================
k2 = kpi.copy()
k2["return_status"] = np.select(
    [k2["return_rate"] <= 0.045 , k2["return_rate"] <= 0.055] ,
    ["GOOD" , "WATCH"] , default = "RISK"
)
k2["margin_status"] = np.select(
    [k2["margin"] >= 0.15 , k2["margin"] >= 0.10] ,
    ["GOOD" , "WATCH"] , default = "RISK"
)
k2 = k2[["region" , "sales" , "returns" , "return_rate" , "margin" , "return_status" , "margin_status"]]

def pill_cell(v):
    if v == "GOOD":
        bg , fg = "#DCFCE7" , "#166534"
    elif v == "WATCH":
        bg , fg = "#FEF3C7" , "#92400E"
    else:
        bg , fg = "#FEE2E2" , "#991B1B"
    return (
        f"background-color: {bg} !important; color: {fg} !important; "
        "font-weight: 800; text-align: center; border-radius: 999px;"
    )

s2 = base_light(k2.style.hide(axis = "index") , "Case 2: KPI status badges (Good/Watch/Risk)")
s2 = (
    s2.format({"sales": "{:,.0f}" , "returns": "{:,.0f}"})
      .format({"return_rate": "{:.1%}" , "margin": "{:.1%}"})
      .map(pill_cell , subset = ["return_status" , "margin_status"])
)
s2

region,sales,returns,return_rate,margin,return_status,margin_status
East,1297,58,4.5%,18.0%,GOOD,GOOD
West,1579,71,4.5%,12.0%,GOOD,WATCH
Central,1321,75,5.7%,21.0%,RISK,GOOD
North,980,22,2.2%,9.0%,GOOD,RISK


In [42]:
#==================================
# Case 3) High-contrast heatmap
#==================================
def heatmap_col(series , c_lo = "#DBEAFE" , c_hi = "#1D4ED8"):
    v = series.astype(float)
    lo , hi = v.min() , v.max()
    denom = (hi - lo) if hi != lo else 1.0
    styles = []
    for x in v:
        t = (x - lo) / denom
        color = hex_lerp(c_lo , c_hi , float(t))
        txt = "#111827" if t < 0.55 else "#FFFFFF"
        styles.append(
            f"background-color: {color} !important; color: {txt} !important; font-weight: 800;"
        )
    return styles

s3 = base_light(kpi.style.hide(axis = "index") , "Case 3: Heatmap scan (fast highs/lows)")
s3 = fmt_kpi(s3)
s3 = s3.apply(heatmap_col , subset = ["sales"] , axis = 0)
s3 = s3.apply(heatmap_col , subset = ["returns"] , axis = 0)
s3 = s3.apply(heatmap_col , subset = ["return_rate"] , axis = 0)
s3 = s3.apply(heatmap_col , subset = ["margin"] , axis = 0)
s3

region,sales,returns,return_rate,margin
East,1297,58,0.044719,0.180000
West,1579,71,0.044965,0.120000
Central,1321,75,0.056775,0.210000
North,980,22,0.022449,0.090000


In [43]:
#=======================================
# Case 4) Progress bars inside cells
#=======================================
def bar_css(series , bar = "#60A5FA"):
    v = series.astype(float)
    lo , hi = v.min() , v.max()
    denom = (hi - lo) if hi != lo else 1.0
    out = []
    for x in v:
        t = (x - lo) / denom
        pct = int(round(100 * float(t)))
        out.append(
            "background: linear-gradient(90deg, "
            f"{bar} {pct}%, #FFFFFF {pct}%) !important; "
            f"color: {hw['text']} !important; font-weight: 800;"
        )
    return out

s4 = base_light(kpi.style.hide(axis = "index") , "Case 4: In-cell progress bars")
s4 = fmt_kpi(s4)
s4 = s4.apply(bar_css , subset = ["sales"] , axis = 0)
s4 = s4.apply(bar_css , subset = ["returns"] , axis = 0)
s4

region,sales,returns,return_rate,margin
East,1297,58,0.044719,0.180000
West,1579,71,0.044965,0.120000
Central,1321,75,0.056775,0.210000
North,980,22,0.022449,0.090000


In [44]:
#===============================
# Case 5) Highlight Exceptions
#===============================
def exceptions(df_):
    out = pd.DataFrame(
        "color: #9CA3AF !important; background-color: #FFFFFF !important;" ,
        index = df_.index , columns = df_.columns
    )

    rr = df_["return_rate"] > 0.05
    mg = df_["margin"] < 0.10
    out.loc[rr , "return_rate"] = "background-color:#FFEDD5 !important; color:#9A3412 !important; font-weight:900;"
    out.loc[mg , "margin"] = "background-color:#FEE2E2 !important; color:#991B1B !important; font-weight:900;"
    return out

s5 = base_light(kpi.style.hide(axis = "index") , "Case 5: Highlight Exceptions")
s5 = fmt_kpi(s5).apply(exceptions , axis = None)
s5

region,sales,returns,return_rate,margin
East,1297,58,0.044719,0.180000
West,1579,71,0.044965,0.120000
Central,1321,75,0.056775,0.210000
North,980,22,0.022449,0.090000


In [45]:
#==================================
# Case 6) MultiIndex mini-report
#==================================
total_row = pd.DataFrame(
    {"sales": [mi["sales"].sum()] , "returns": [mi["returns"].sum()]} ,
    index = pd.MultiIndex.from_tuples([("ALL" , "ALL" , "TOTAL")] , names = mi.index.names)
)
report = pd.concat([total_row , mi])

def style_total(df_):
    out = pd.DataFrame("" , index = df_.index , columns = df_.columns)

    out.iloc[0 , :] = "background-color:#111827 !important; color:#FFFFFF !important; font-weight:950; text-transform:uppercase;"

    out.iloc[1::2 , :] = f"background-color:{hw['bg']} !important;"
    out.iloc[2::2 , :] = f"background-color:{hw['panel']} !important;"
    return out

s6 = base_light(report.style , "Case 6: MultiIndex mini-report")
s6 = (
    s6.format({"sales": "{:,.0f}" , "returns": "{:,.0f}"})
      .format_index(lambda x: x.upper() , level = "Product")
      .format_index(lambda x: f"[{x}]" , level = "Month").apply(style_total , axis = None)
)
s6